In [ ]:
import pandas as pd # pandas dataframes
from sklearn.ensemble import RandomForestClassifier # model that takes in data to train and classify the imports
from sklearn.metrics import f1_score, make_scorer, classification_report 
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder # one hot encoding of team names
import numpy as np
from collections import defaultdict
import time


In [ ]:
# import csv files from git repository
teamPath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/teamData.csv'
playerPath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/playerData.csv'
outcomePath = '/Users/lorneez/projects/sports_predictor/Sports_Model/Data/outcomeData.csv'

In [ ]:
# convert csv files into pandas dataframes
teamData = pd.read_csv(teamPath)
playerData = pd.read_csv(playerPath, engine='python')
outcomeData = pd.read_csv(outcomePath)

In [ ]:
teamData

### Format All Data

In [ ]:
# drop unwanted columns
outcomeData = outcomeData.drop(["Start Time", "Box Score", "Notes"], axis=1)

# change OT to integer value for number of OT rounds
outcomeData.loc[(outcomeData["OT?"].isna()),'OT?']= 0
outcomeData.loc[(outcomeData["OT?"] == "OT"),'OT?']= 1
outcomeData.loc[(outcomeData["OT?"] == "2OT"),'OT?']= 2
outcomeData.loc[(outcomeData["OT?"] == "3OT"),'OT?']= 3
outcomeData.loc[(outcomeData["OT?"] == "4OT"),'OT?']= 4


In [ ]:
# function for redefining team names
def redefineTeamNames(dataframe):
    
    dataframe = dataframe.replace('Seattle SuperSonics', 'Oklahoma City Thunder', regex=True)
    
    dataframe = dataframe.replace('New Orleans/Oklahoma City Hornets', 'New Orleans Pelicans', regex=True)

    dataframe = dataframe.replace('New Orleans Hornets', 'New Orleans Pelicans', regex=True)
    
    dataframe = dataframe.replace('Charlotte Bobcats', 'Charlotte Hornets', regex=True)
    
    dataframe = dataframe.replace('New Jersey Nets', 'Brooklyn Nets', regex=True)

    return dataframe

In [ ]:
# call function to redefine team names
outcomeData = redefineTeamNames(outcomeData)
teamData = redefineTeamNames(teamData)

# set up encoder and use one hot encoding to transform team names
encoding = LabelEncoder()
encoding.fit(outcomeData['Home'].values)
outcomeData['HomeEncoded'] = encoding.transform(outcomeData['Home'].values)
outcomeData['VisitorEncoded'] = encoding.transform(outcomeData['Visitor'].values)

In [ ]:
# generate column names
teamDataColumn = teamData.drop(['Team', "Year", "Arena", "Attend.", "Attend./G", "G_PG", "MP_PG"], axis=1).columns
print(teamDataColumn)
testTeamDataColumn = teamData.drop(teamData.columns.difference(["3P_PG", "FGA_PG", "FG%_PG", "3P_PG", 
                                                               "3PA_PG", "3P%_PG", "ORB_PG", "DRB_PG", "TRB_PG", "AST_PG"
                                                                , "STL_PG", "BLK_PG", "TOV_PG", "2P_PG", "2PA_PG"]), axis=1).columns
print(testTeamDataColumn)
# outcomeData = outcomeData.loc[outcomeData['Year'] == 2020]
outcomeData



### Feature Engineering

#### y data

In [ ]:
# function for finding if the home team is the winner
def homeWin(dataframe):
    if dataframe['Winner'] == dataframe['Home']:
        return True
    else:
        return False
    
# add in HomeWin column
outcomeData['HomeWin'] = outcomeData.apply(homeWin, axis=1)

In [ ]:
won_last = defaultdict(int)

for index, row in outcomeData.iterrows():
    home_team = row["Home"]
    visitor_team = row["Visitor"]
    outcomeData.loc[index,'HomeLastWin'] = won_last[home_team]
    outcomeData.loc[index,'VisitorLastWin'] = won_last[visitor_team]
    won_last[home_team] = row["HomeWin"]
    won_last[visitor_team] = not row["HomeWin"]

In [ ]:
outcomeData["HomeWinStreak"] = 0
outcomeData["VisitorWinStreak"] = 0

win_streak = defaultdict(int)

for index, row in outcomeData.iterrows():
    home_team = row["Home"]
    visitor_team = row["Visitor"]
    outcomeData.loc[index,'HomeWinStreak'] = win_streak[home_team]
    outcomeData.loc[index,'VisitorWinStreak'] = win_streak[visitor_team]
    if row["HomeWin"]:
        win_streak[home_team] += 1
        win_streak[visitor_team] = 0
    else:
        win_streak[home_team] = 0
        win_streak[visitor_team] += 1

In [ ]:
outcomeData

In [ ]:
def home_team_ranks_higher(row):
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    home_rank = ladder.loc[home_team]["Rk"]
    visitor_rank = ladder.loc[visitor_team]["Rk"]
    return home_rank < visitor_rank

In [ ]:
def home_team_won_last(row):
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    
    teams = tuple(sorted([home_team, visitor_team]))
    result = 1 if last_match_winner[teams] == row["Home Team"] else 0
    
    winner = row["Home Team"] if row["HomeWin"] else row["Visitor Team"]
    last_match_winner[teams] = winner
    
    return result
    
outcomeData['HomeTeamWonLast'] = outcomeData.apply(home_team_won_last, axis=1)

In [ ]:
# spread
outcomeData["Final Home Spread"] = (outcomeData["VisitorPTS"] - outcomeData["HomePTS"]).astype(float)

#### x data

In [ ]:
def differentialVariableGenerator(outcomeData, teamData, teamDataColumn):
                                  
    global teamData_global
    global counter 
    global differentialFeatureData
    x = []
    for col in teamDataColumn:
        x.append(col + "_home")
        x.append(col + "_visitor")
    
    differentialFeatureData = pd.DataFrame(index=range(outcomeData.shape[0]), columns = x)
    print(outcomeData.shape[0])
    teamData_global = teamData
    totalCount = outcomeData.shape[0]
    counter = 0
    overallTime = time.time()
    
    def calcDifferences(row):
                                  
        global teamData_global
        global counter
                                  
        counter = counter + 1;
        start = time.time()
        year = row['Year']
        visitor = row['Visitor']
        home = row['Home']
                                  
        def columnCalc(col):
                                  
            variable = col.name
            var_home = variable+"_home"
            var_visitor = variable+"_visitor"
            year_subset = teamData_global.loc[(teamData_global.Year == year)]
            
            visitorValue = year_subset.loc[year_subset.Team.astype(str).str.contains(visitor)][variable]
            homeValue = year_subset.loc[year_subset.Team.astype(str).str.contains(home)][variable]
            
            differentialFeatureData[var_home][counter-1] = homeValue.iloc[0]
            differentialFeatureData[var_visitor][counter-1] = visitorValue.iloc[0]
                                  
        teamData_global[teamDataColumn].apply(columnCalc)
        #print(differentialFeatureData)
        print(visitor, home, time.time() - start,time.time() - overallTime)
        print("{count}/{total}".format(count = counter, total = totalCount))
                                  
    outcomeData.apply(calcDifferences, axis=1)
    print("Total time {time} seconds".format(time = time.time() - overallTime))
    return differentialFeatureData

In [ ]:
testOutcomeData = differentialVariableGenerator(outcomeData, teamData, testTeamDataColumn)

In [ ]:
testOutcomeData.to_csv('/Users/lorneez/projects/sports_predictor/Sports_Model/Data/feature_data.csv')
testOutcomeData

In [ ]:
x_columns = []
for col in teamDataColumn:
    x_columns.append(col + "_DIFF")
print(x_columns)

### Training Model

In [ ]:
# target column (in this case, whether or not the home team will win)
y = outcomeData["HomeWin"]
# y = outcomeData["Final Home Spread"]

# columns that will be used to make the prediction
# X = outcomeData[x_columns]
# X = testOutcomeData
X_previous_wins = outcomeData[["HomeLastWin", "VisitorLastWin"]].values
X_win_streak = outcomeData[["HomeLastWin", "VisitorLastWin", "HomeWinStreak", "VisitorWinStreak"]].values

# X = outcomeData[["HomeLastWin", "VisitorLastWin", "HomeTeamRanksHigher", "HomeTeamWonLast"]].values

# set up scorer for testing accuracy
scorer = make_scorer(f1_score, pos_label = None, average = "weighted")

In [ ]:
# Set up train test split
# X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(X_1, y, test_size=0.20, random_state=100)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=100)

In [ ]:
y_true = outcomeData["HomeWin"].values

# cross val score
clf = RandomForestClassifier(random_state = 14)
#scores = cross_val_score(clf, X_train, y_train, scoring=scorer)
scores_previous_wins = cross_val_score(clf, X_previous_wins, y_true, scoring=scorer)
scores_win_streak = cross_val_score(clf, X_win_streak, y_true, scoring=scorer)

In [ ]:
# mean score
y_pred = [1] * len(y_true)

print("F1 Score to Beat: {:.4f}".format(f1_score(y_true, y_pred, pos_label=None, average='weighted')))

print("F1 Score with Previous Wins: {:.4f}".format(np.mean(scores_previous_wins)))

print("F1 Score with Win Streaks: {:.4f}".format(np.mean(scores_win_streak)))
